In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import torchvision 
import torchvision.transforms as transforms 
import numpy as np 
from torch.utils.data import DataLoader
from scipy.stats import truncnorm

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
device

device(type='cuda')

In [ ]:
class CNN(nn.Module):
    def __init__(self, config, num_classes=10):
        super(CNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = 1
        for i in range(config['num_conv_layers']):
            out_channels = config['filters'][i]
            kernel_size = config['filter_sizes'][i]
            padding = kernel_size // 2  
            conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
            self.layers.append(conv)
            if config['conv_activations'][i] == 1:  # relu
                self.layers.append(nn.ReLU())
            elif config['conv_activations'][i] == 2:  # tanh 
                self.layers.append(nn.Tanh())
            elif config['conv_activations'][i] == 3:  # linear 
                self.layers.append(nn.Identity())
            elif config['conv_activations'][i] == 4:  # sigmoid
                self.layers.append(nn.Sigmoid())
            # add pooling if specified aur agar dimension 0 na ho gaya output ki
            if config['pooling'][i]:
                pool_size = min(config['pool_sizes'][i], 3) # limit pooling size 
                self.layers.append(nn.MaxPool2d(pool_size))
            in_channels = out_channels
        self.layers.append(nn.Flatten())

        # calculate flattened size 
        with torch.no_grad():
            x = torch.zeros(1, 1, 28, 28)
            for layer in self.layers:
                x = layer(x)
                if len(x.shape) > 2 and (x.shape[-1] == 0 or x.shape[-2] == 0):
                    raise ValueError("Invalid architecture - feature maps too small")
            flattened_size = x.shape[1]
				# build dense layers 
        for i in range(config['num_dense_layers']):
            out_features = config['dense_units'][i]
            dense = nn.Linear(flattened_size if i == 0 else config['dense_units'][i-1], out_features)
            self.layers.append(dense)

            # add activation except for the  last layer 
            if i < config['num_dense_layers'] - 1:
                if config['dense_activations'][i] == 1:  
                    self.layers.append(nn.ReLU())
                elif config['dense_activations'][i] == 2:  
                    self.layers.append(nn.Tanh())
                elif config['dense_activations'][i] == 3:  
                    self.layers.append(nn.Identity())
                elif config['dense_activations'][i] == 4: 
                    self.layers.append(nn.Sigmoid())

            flattened_size = out_features

        # output layer
        self.output_layer = nn.Linear(flattened_size, num_classes)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.output_layer(x)
        return x

In [5]:
class CSA:
    def __init__(self, population_size=10, clone_factor=0.3, mutation_rate=0.1, num_generations=5, dataset_name='digits'):
        self.population_size = population_size
        self.clone_factor = clone_factor
        self.mutation_rate = mutation_rate
        self.num_generations = num_generations
        self.dataset_name = dataset_name

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        self.train_dataset = torchvision.datasets.EMNIST(root='./data', split=dataset_name,train=True, transform=transform, download=True)
        self.test_dataset = torchvision.datasets.EMNIST(root='./data', split=dataset_name,train=False, transform=transform)
        self.num_classes = len(self.train_dataset.classes)

        self.hyperparameter_ranges = {
            'num_epochs': (1, 49),  
            'batch_size': (1, 255),
            'num_conv_layers': (1, 9), 
            'filters': (1, 65),
            'filter_sizes': (1,9),  
            'conv_activations': (1, 4),  # 1: ReLU, 2: Tanh, 3: Linear, 4: Sigmoid
            'pooling': (1, 2),  # 1: False, 2: True
            'pool_sizes': (2, 9),
            'num_dense_layers': (1, 9),  # Reduced for simpler architectures
            'dense_units': (1, 65),
            'dense_activations': (1, 4), 
            'optimizer': (1, 5),  # 1: SGD, 2: Adadelta, 3: RMSprop, 4: Adam, 5: Nadam
            'learning_rate': [0.0001, 0.001, 0.01]
        }

    def initialize_antibody(self):
        config = {}
        config['num_epochs'] = np.random.randint(*self.hyperparameter_ranges['num_epochs'])
        config['batch_size'] = np.random.randint(*self.hyperparameter_ranges['batch_size'])
        config['num_conv_layers'] = np.random.randint(*self.hyperparameter_ranges['num_conv_layers'])
        config['filters'] = [np.random.randint(*self.hyperparameter_ranges['filters']) for _ in range(config['num_conv_layers'])]
        config['filter_sizes'] = [np.random.choice([3, 5]) for _ in range(config['num_conv_layers'])] # 3 to 5 co time boht lay raha tha 
        config['conv_activations'] = [np.random.randint(*self.hyperparameter_ranges['conv_activations']) for _ in range(config['num_conv_layers'])]
        config['pooling'] = [np.random.randint(*self.hyperparameter_ranges['pooling']) == 2 for _ in range(config['num_conv_layers'])]
        config['pool_sizes'] = [np.random.randint(*self.hyperparameter_ranges['pool_sizes']) for _ in range(config['num_conv_layers'])]
        config['num_dense_layers'] = np.random.randint(*self.hyperparameter_ranges['num_dense_layers'])
        config['dense_units'] = [np.random.randint(*self.hyperparameter_ranges['dense_units']) for _ in range(config['num_dense_layers'])]
        config['dense_activations'] = [np.random.randint(*self.hyperparameter_ranges['dense_activations']) for _ in range(config['num_dense_layers'])]
        config['optimizer'] = np.random.randint(*self.hyperparameter_ranges['optimizer'])
        config['learning_rate'] = np.random.choice(self.hyperparameter_ranges['learning_rate'])

        return config

    def evaluate_antibody(self, config): # train cnn return test accuracy 
        try:
            model = CNN(config, self.num_classes).to(device)
            if config['optimizer'] == 1:
                optimizer = optim.SGD(model.parameters(), lr=config['learning_rate'])
            elif config['optimizer'] == 2:
                optimizer = optim.Adadelta(model.parameters(), lr=config['learning_rate'])
            elif config['optimizer'] == 3:
                optimizer = optim.RMSprop(model.parameters(), lr=config['learning_rate'])
            elif config['optimizer'] == 4:
                optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
            elif config['optimizer'] == 5:
                optimizer = optim.NAdam(model.parameters(), lr=config['learning_rate'])

            criterion = nn.CrossEntropyLoss()

            train_loader = DataLoader(self.train_dataset, batch_size=config['batch_size'], shuffle=True)
            test_loader = DataLoader(self.test_dataset, batch_size=config['batch_size'], shuffle=False)

            best_accuracy = 0.0
            for epoch in range(config['num_epochs']):
                model.train()
                for images, labels in train_loader:
                    images = images.to(device)
                    labels = labels.to(device)

                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                model.eval()
                with torch.no_grad():
                    correct = 0
                    total = 0
                    for images, labels in test_loader:
                        images = images.to(device)
                        labels = labels.to(device)
                        outputs = model(images)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()

                    accuracy = correct / total
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy

            return best_accuracy
        except (ValueError, RuntimeError) as e:
            # low accuracy for invalid architectures
            print(f"Invalid architecture encountered: {e}")
            return 0.0

    def mutate_antibody(self, antibody, affinity): # mutation using truchnorm 
        mutated = antibody.copy()

        mutation_prob = np.exp(-self.mutation_rate * (1 - affinity))

        for key in mutated:
            if key in ['filters', 'filter_sizes', 'conv_activations', 'pooling', 'pool_sizes', 'dense_units', 'dense_activations']:
                for i in range(len(mutated[key])):
                    if np.random.random() < mutation_prob:
                        if key == 'filter_sizes':
                            mutated[key][i] = np.random.choice([3, 5])  # Only 3x3 or 5x5 for simplicity
                        elif key == 'pooling':
                            mutated[key][i] = not mutated[key][i] 
                        else:
                            current = mutated[key][i]
                            lb, ub = self.hyperparameter_ranges[key]
                            mutated[key][i] = self._truncated_gaussian_mutation(current, lb, ub)
            elif key == 'learning_rate':
                if np.random.random() < mutation_prob:
                    mutated[key] = np.random.choice(self.hyperparameter_ranges[key])
            elif key in self.hyperparameter_ranges:
                if np.random.random() < mutation_prob:
                    current = mutated[key]
                    lb, ub = self.hyperparameter_ranges[key]
                    mutated[key] = self._truncated_gaussian_mutation(current, lb, ub)
        return mutated

    def _truncated_gaussian_mutation(self, current, lb, ub, scale=0.1):
        mean = current
        std = scale * (ub - lb)

        a = (lb - mean) / std
        b = (ub - mean) / std

        new_val = truncnorm.rvs(a, b, loc=mean, scale=std, size=1)[0]

        if isinstance(lb, int) and isinstance(ub, int):
            new_val = int(round(new_val))

        # clip
        new_val = max(lb, min(ub, new_val))

        return new_val

    def run(self):
        # initial population 
        population = [self.initialize_antibody() for _ in range(self.population_size)]
        # evaluate population initial 
        affinities = []
        for antibody in population:
            accuracy = self.evaluate_antibody(antibody)
            affinities.append(accuracy)
            print(f"Initial antibody accuracy: {accuracy:.4f}")

        best_affinity = max(affinities)
        best_antibody = population[np.argmax(affinities)]

        print(f"\nInitial best affinity: {best_affinity:.4f}")

        # evo 
        for generation in range(self.num_generations):
            print(f"\nGeneration {generation + 1}/{self.num_generations}")
						# selection 
            selected = []
            selected_affinities = []
            for antibody, affinity in zip(population, affinities):
                if affinity >= best_affinity * 0.9: 
                    selected.append(antibody)
                    selected_affinities.append(affinity)

            if not selected:
                print("No antibodies better than current best, keeping population")
                selected = population.copy()
                selected_affinities = affinities.copy()

            # create clones proportional to affinity
            clones = []
            num_clones = max(1,int(self.clone_factor * self.population_size))
            if sum(selected_affinities) > 0:
              norm_affinities = np.array(selected_affinities) / sum(selected_affinities)
            else:
              norm_affinities = np.ones(len(selected_affinities)) / len(selected_affinities)

            clone_counts = np.random.multinomial(num_clones, norm_affinities)

            for antibody, count in zip(selected, clone_counts):
                clones.extend([antibody.copy() for _ in range(count)])

            # mutation 
            mutated_clones = []
            for clone in clones:
                # find clone origin 
                idx = selected.index(next(ab for ab in selected if ab == clone))
                affinity = selected_affinities[idx]

                mutated_clone = self.mutate_antibody(clone, affinity)
                mutated_clones.append(mutated_clone)

            # eval mutated classes 
            clone_affinities = []
            valid_clones = []
            for clone in mutated_clones:
                accuracy = self.evaluate_antibody(clone)
                if accuracy > 0:
                  clone_affinities.append(accuracy)
                  valid_clones.append(clone)
                  print(f"Clone accuracy: {accuracy:.4f}")

            if not clone:
              print("All clones were invalid, keeping previous best")
              best_clone_affinity = best_affinity
              best_clone = best_antibody
            else:
              best_clone_idx = np.argmax(clone_affinities)
              best_clone_affinity = clone_affinities[best_clone_idx]
              best_clone = mutated_clones[best_clone_idx]

            # replace if clone is better 
            if best_clone_affinity > best_affinity:
                best_affinity = best_clone_affinity
                best_antibody = best_clone
                print(f"New best affinity: {best_affinity:.4f}")

            # new population: best clone + random antibodies
            new_population = [best_clone]
            new_population.extend([self.initialize_antibody() for _ in range(self.population_size - 1)])

            # eval new population 
            population = new_population
            new_affinities = [best_clone_affinity]
            for ab in new_population[1:]:
                accuracy = self.evaluate_antibody(ab)
                new_affinities.append(accuracy)
                print(f"New antibody accuracy: {accuracy:.4f}")

            affinities = new_affinities

            avg_affinity = np.mean(affinities)
            print(f"Average affinity: {avg_affinity:.4f}, Best affinity: {best_affinity:.4f}")

        return best_antibody, best_affinity

In [ ]:
....

In [6]:
csa = CSA(population_size=10, clone_factor=0.3, mutation_rate=0.1, num_generations=3, dataset_name='digits') 
bestconfig, bacc =csa.run() 
print('Best configuration found: ') 
for key, value in bestconfig.items(): 
    print(f"{key}: {value}")
print(f"Best Accuracy : {bacc}")


100%|██████████| 562M/562M [00:31<00:00, 17.9MB/s]
Initial antibody accuracy: 0.1000
Initial antibody accuracy: 0.9919
Initial antibody accuracy: 0.1000
Initial antibody accuracy: 0.1013
Initial antibody accuracy: 0.1000
Initial antibody accuracy: 0.9940
Initial antibody accuracy: 0.9261
Initial antibody accuracy: 0.9854
Initial antibody accuracy: 0.9554
Initial antibody accuracy: 0.1000

Initial best affinity: 0.9940

Generation 1/3
Clone accuracy: 0.9889
Clone accuracy: 0.9917
Clone accuracy: 0.9323
New antibody accuracy: 0.9871
New antibody accuracy: 0.9906
New antibody accuracy: 0.9906
New antibody accuracy: 0.5907
New antibody accuracy: 0.9681
New antibody accuracy: 0.1000



In [9]:
csa = CSA(population_size=2, clone_factor=0.3, mutation_rate=0.1, num_generations=3, dataset_name='digits') 
bestconfig, bacc =csa.run() 
print('Best configuration found: ') 
for key, value in bestconfig.items(): 
    print(f"{key}: {value}")
print(f"Best Accuracy : {bacc}")


Initial antibody accuracy: 0.9768
Initial antibody accuracy: 0.9868

Initial best affinity: 0.9868

Generation 1/3
Clone accuracy: 0.9912
New best affinity: 0.9912
New antibody accuracy: 0.9765
Average affinity: 0.9839, Best affinity: 0.9912

Generation 2/3
Clone accuracy: 0.9932
New best affinity: 0.9932
New antibody accuracy: 0.9727
Average affinity: 0.9829, Best affinity: 0.9932

Generation 3/3
Clone accuracy: 0.9407
New antibody accuracy: 0.1000
Average affinity: 0.5204, Best affinity: 0.9932

Best configuration found:
num_epochs: 9
batch_size: 170
num_conv_layers: 2
filters: [26, 47]
filter_sizes: [3, 5]
conv_activations: [3, 3]
pooling: [False, False]
pool_sizes: [3, 3]
num_dense_layers: 2
dense_units: [44, 112]
dense_activations: [3, 2]
optimizer: 3
learning_rate: 0.001

Best accuracy: 0.9932



In [ ]:
csa = CSA(population_size=3, clone_factor=0.3, mutation_rate=0.1, num_generations=2, dataset_name='digits') 
bestconfig, bacc =csa.run() 
print('Best configuration found: ') 
for key, value in bestconfig.items(): 
    print(f"{key}: {value}")
print(f"Best Accuracy : {bacc}")

"
Initial antibody accuracy: 0.1000
Initial antibody accuracy: 0.9926
Initial antibody accuracy: 0.9937

Initial best affinity: 0.9937

Generation 1/2
Clone accuracy: 0.9926
New antibody accuracy: 0.9798
New antibody accuracy: 0.9889
Average affinity: 0.9871, Best affinity: 0.9937

Generation 2/2
Clone accuracy: 0.8450
New antibody accuracy: 0.9703
New antibody accuracy: 0.9463
Average affinity: 0.9205, Best affinity: 0.9937

Best configuration found:
num_epochs: 6
batch_size: 84
num_conv_layers: 4
filters: [29, 53, 56, 60]
filter_sizes: [5, 3, 3, 3]
conv_activations: [1, 2, 1, 3]
pooling: [True, True, False, False]
pool_sizes: [3, 2, 3, 2]
num_dense_layers: 1
dense_units: [48]
dense_activations: [3]
optimizer: 4
learning_rate: 0.0001

Best accuracy: 0.9937
      

